<a href="https://colab.research.google.com/github/yufengg/advent-of-code-2024/blob/master/AoC_2024_day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: python code download advent of code file

import requests
import os
from datetime import datetime

def download_aoc_input(year, day, session_cookie):
  """Downloads the input file for a specific Advent of Code day.

  Args:
    year: The year of the challenge (e.g., 2023).
    day: The day of the challenge (e.g., 1).
    session_cookie: Your session cookie from the Advent of Code website.
  """

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  headers = {
      "User-Agent": "github.com/yourusername/yourrepo (or your email)",  # Replace with your info
      "Cookie": f"session={session_cookie}"
  }

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Create a directory for the year if it doesn't exist
    os.makedirs(str(year), exist_ok=True)

    filename = os.path.join(str(year), f"day{day}.txt")
    with open(filename, "w") as file:
      file.write(response.text)
    print(f"Successfully downloaded input for year {year}, day {day} to {filename}")
    return filename

  except requests.exceptions.RequestException as e:
    print(f"Error downloading input: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Example usage:  Replace with your actual values
year = 2024  #@param {type:"integer"}
day = 5 #@param {type:"integer"}
session_cookie = "53616c7465645f5fd6809f86933785dcec3c4f79b521758a796b0808898e32ff9a09d06dd15282b2a68340b7bf970ab66894fa0bf9271caa521bc5d5fc8df3b9" #@param {type:"string"}

input_filename = download_aoc_input(year, day, session_cookie)

Successfully downloaded input for year 2024, day 5 to 2024/day5.txt


In [3]:
import jax
import jax.numpy as jnp

In [4]:
# need to split the file in half

In [5]:

def read_input(filename, parser=None):
  with open(filename, 'r') as file:
    lines = file.read().splitlines()
    if parser:
      lines = list(map(parser, lines))
  return lines


In [6]:

def read_input(filename, parser1, parser2):
  with open(filename, 'r') as file:
    lines = file.readlines()
    part1 = []
    part2 = []
    in_part_1 = True
    for line in lines:
      if line == '\n':
        in_part_1 = False
        continue
      if in_part_1:
        part1.append(parser1(line))
      else:
        part2.append(parser2(line))

  return part1, part2


In [11]:
input_filename = f'{year}/day{day}.txt'
def parser1(line):
  return tuple(map(int, line.split('|')))

def parser2(line):
  return list(map(int, line.split(',')))


# Test data

In [36]:
test_data = '''47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
'''
with open('day5_test.txt', 'w') as file:
  lines = file.writelines(test_data)


In [151]:
test_rules, test_pages = read_input('day5_test.txt', parser1, parser2)


In [152]:
sum([check_rules(p, test_rules) for p in test_pages])


success: [75, 47, 61, 53, 29]
success: [97, 61, 53, 29, 13]
success: [75, 29, 13]


143

# Part 1

In [122]:
rules, pages = read_input(input_filename, parser1, parser2)


In [105]:
rules[:10]

[(53, 62),
 (43, 48),
 (43, 99),
 (62, 76),
 (62, 19),
 (62, 85),
 (58, 55),
 (58, 84),
 (58, 43),
 (58, 64)]

In [106]:
pages[:10]

[[71, 31, 35, 38, 58, 83, 43, 67, 11, 45, 32],
 [46, 64, 27, 32, 48, 98, 62, 99, 14],
 [53, 55, 27, 11, 45, 93, 32, 48, 98, 51, 86],
 [97, 76, 73, 34, 69, 47, 16, 49, 87, 92, 71, 39, 31, 35, 17, 38, 53, 57, 83],
 [59, 17, 58, 55, 46, 27, 84, 43, 67, 64, 45, 93, 32, 48, 98, 51, 86],
 [83,
  46,
  67,
  64,
  27,
  43,
  36,
  32,
  55,
  77,
  86,
  89,
  45,
  51,
  93,
  98,
  62,
  85,
  23,
  61,
  91],
 [46, 67, 93, 48, 51],
 [43, 67, 64, 11, 89, 32, 98, 51, 86, 77, 85, 23, 91, 99, 97, 66, 76],
 [92, 82, 71, 39, 31, 35, 59, 17, 57, 83, 27, 43, 67],
 [49, 87, 92, 82, 71, 31, 59, 17, 57, 83, 55, 46, 84, 43, 11]]

at each point in the pages, we need to be able to look up other values

For any given number X, there are rules where it is present in the first spot (i.e. "X|Y"), and other rules where it is present in the 2nd spot (i.e. "Y|X").

*[choosing this one]* One option would be to look at all rules for a given page number X (which requires looking for the presence of various page number pairs throughout the rules, of which there would be N-1 rules to check for each X, so O(N^2) where N is max=23 pages).

The other direction would be to look at every potentially matching rule for a given page number (X|Y and Y|X), and check for their presence in the pages list. This seems worse.

need to be able to search for rule pairs; can probably just shove it into a set.

note that when searching, need to search for the "broken" version of the rule that a given page pair exhibits. That is, if X and Y obey X|Y, search for Y|X, as if that exists, then it's no good. The presence of X|Y does nothing for us

In [107]:
rules_set = set(rules)

In [123]:
def check_rules(pages, rules_to_check=rules_set):
  for i, X in enumerate(pages):
    for j, Y in enumerate(pages):
      if i == j:
        continue
      # if (X,Y) in rules_set:
      #   continue
      # print(f"X: {X}, i: {i}, Y: {Y}, j: {j}")

      # if i<j: # normal ordering
      check_me = (Y,X)
      if i>j: # reverse ordering
        check_me = (X,Y)
      # print(check_me)

      if check_me in rules_to_check:
        # print(f"X: {X}, Y: {Y}")
        return False

  print(f"success: {pages}")
  # find the middle index
  return pages[len(pages)//2]
  # return True

In [124]:
[check_rules(p, test_rules) for p in test_pages]

success: [75, 47, 61, 53, 29]
success: [97, 61, 53, 29, 13]
success: [75, 29, 13]


[61, 53, 29, False, False, False]

In [58]:
# print(pages[6])
check_rules(pages[6])

success: [46, 67, 93, 48, 51]


True

In [125]:
good_pages = [check_rules(p) for p in pages]

success: [71, 31, 35, 38, 58, 83, 43, 67, 11, 45, 32]
success: [53, 55, 27, 11, 45, 93, 32, 48, 98, 51, 86]
success: [97, 76, 73, 34, 69, 47, 16, 49, 87, 92, 71, 39, 31, 35, 17, 38, 53, 57, 83]
success: [59, 17, 58, 55, 46, 27, 84, 43, 67, 64, 45, 93, 32, 48, 98, 51, 86]
success: [46, 67, 93, 48, 51]
success: [43, 67, 64, 11, 89, 32, 98, 51, 86, 77, 85, 23, 91, 99, 97, 66, 76]
success: [49, 87, 92, 82, 71, 31, 59, 17, 57, 83, 55, 46, 84, 43, 11]
success: [59, 17, 58, 53, 57, 83, 55, 36, 84, 43, 67, 11, 89, 45, 93, 32, 48, 51, 86]
success: [39, 31, 35, 59, 38, 58, 53, 57, 83, 55, 36, 46, 27, 84, 43, 67, 64, 11, 89, 45, 93, 32, 48]
success: [38, 57, 83, 55, 67, 32, 48, 62, 77]
success: [84, 43, 67, 64, 89, 45, 32, 48, 51, 86, 85, 61, 19, 99, 66]
success: [97, 76, 73, 34, 16, 49, 87, 92, 71, 38, 58, 83, 55]
success: [19, 14, 97, 76, 73, 34, 69, 87, 82, 71, 39, 59, 38, 58, 53]
success: [61, 91, 19, 14, 97, 66, 76, 73, 69, 47, 16, 49, 87, 92, 82, 71, 39, 31, 35, 17, 38]
success: [69, 47, 92

In [126]:
sum(good_pages)

7307

In [127]:
len([p for p in good_pages if p is False])

82

# Part 2

- extract the unsuccessful ones
- apply new rules func that swaps erroneous placements
- assume (for now) that we only need a single pass, and that swaps will not introduce more errors to fix



In [128]:
def get_bad_pages(pages, rules_to_check=rules_set):
  for i, X in enumerate(pages):
    for j, Y in enumerate(pages):
      if i == j:
        continue
      # if (X,Y) in rules_set:
      #   continue
      # print(f"X: {X}, i: {i}, Y: {Y}, j: {j}")

      # if i<j: # normal ordering
      check_me = (Y,X)
      if i>j: # reverse ordering
        check_me = (X,Y)
      # print(check_me)

      if check_me in rules_to_check:
        # print(f"X: {X}, Y: {Y}")
        return pages

In [129]:
bad_pages_all = [ get_bad_pages(p) for p in pages]

In [131]:
bad_pages = []
for p in bad_pages_all:
  if p is not None:
    bad_pages.append(p)

In [132]:
bad_pages[:10]

[[32, 99, 27, 62, 48, 64, 46, 98, 14],
 [89,
  91,
  55,
  62,
  98,
  93,
  51,
  45,
  23,
  85,
  83,
  86,
  43,
  32,
  27,
  64,
  67,
  36,
  61,
  46,
  77],
 [83, 82, 27, 59, 57, 17, 92, 43, 35, 71, 39, 67, 31],
 [55, 19, 98, 61, 23, 32, 91],
 [84,
  43,
  77,
  48,
  86,
  62,
  89,
  53,
  85,
  57,
  98,
  64,
  36,
  93,
  11,
  58,
  55,
  83,
  51,
  27,
  45,
  46,
  67],
 [61, 32, 85, 27, 19, 23, 48],
 [55, 98, 57, 67, 38, 51, 11],
 [99, 92, 73, 49, 51, 91, 82, 97, 71, 47, 34, 76, 19, 61, 77, 66, 87, 86, 69],
 [59, 17, 38, 93, 43, 83, 89, 58, 86, 11, 64, 84, 32, 45, 57],
 [19, 35, 97, 85, 66, 77, 49, 92, 34, 91, 87]]

In [166]:
def swap_rules(pages_ext, rules_to_check=rules_set):
  pages = pages_ext.copy()
  print(f"checking: {pages}")
  for i, X in enumerate(pages):
    for j, Y in enumerate(pages):
      if i == j:
        continue
      # if (X,Y) in rules_set:
      #   continue
      # print(f"X: {X}, i: {i}, Y: {Y}, j: {j}")

      # if i<j: # normal ordering
      check_me = (Y,X)
      if i>j: # reverse ordering
        check_me = (X,Y)
      # print(check_me)

      if check_me in rules_to_check:
        print(f"[SWAP] X: {X}, i: {i}, Y: {Y}, j: {j}")
        # print(f"X: {X}, Y: {Y}")
        # need to swap X and Y and positions i and j
        pages[i], pages[j] = pages[j], pages[i]
        return swap_rules(pages, rules_to_check)

  print(f"finished: {pages}")
  return pages[len(pages)//2]


In [167]:
temp_pages = test_pages.copy()
[swap_rules(p, test_rules) for p in temp_pages[3:]]


checking: [75, 97, 47, 61, 53]
[SWAP] X: 75, i: 0, Y: 97, j: 1
checking: [97, 75, 47, 61, 53]
finished: [97, 75, 47, 61, 53]
checking: [61, 13, 29]
[SWAP] X: 13, i: 1, Y: 29, j: 2
checking: [61, 29, 13]
finished: [61, 29, 13]
checking: [97, 13, 75, 29, 47]
[SWAP] X: 13, i: 1, Y: 75, j: 2
checking: [97, 75, 13, 29, 47]
[SWAP] X: 13, i: 2, Y: 29, j: 3
checking: [97, 75, 29, 13, 47]
[SWAP] X: 29, i: 2, Y: 47, j: 4
checking: [97, 75, 47, 13, 29]
[SWAP] X: 13, i: 3, Y: 29, j: 4
checking: [97, 75, 47, 29, 13]
finished: [97, 75, 47, 29, 13]


[47, 29, 47]

In [153]:
temp_pages = test_pages.copy()
[swap_rules(p, test_rules) for p in temp_pages[3:]]


checking: [75, 97, 47, 61, 53]
[SWAP] X: 75, i: 0, Y: 97, j: 1
checking: [61, 13, 29]
[SWAP] X: 13, i: 1, Y: 29, j: 2
checking: [97, 13, 75, 29, 47]
[SWAP] X: 13, i: 1, Y: 75, j: 2
[SWAP] X: 13, i: 1, Y: 29, j: 3
[SWAP] X: 13, i: 1, Y: 47, j: 4
[SWAP] X: 13, i: 2, Y: 75, j: 3
[SWAP] X: 13, i: 2, Y: 29, j: 4
[SWAP] X: 13, i: 3, Y: 75, j: 4


[47, 29, 29]

In [136]:
test_pages

[[75, 47, 61, 53, 29],
 [97, 61, 53, 29, 13],
 [75, 29, 13],
 [97, 75, 47, 61, 53],
 [61, 29, 13],
 [97, 47, 29, 75, 13]]

In [137]:
temp_pages

[[75, 47, 61, 53, 29],
 [97, 61, 53, 29, 13],
 [75, 29, 13],
 [97, 75, 47, 61, 53],
 [61, 29, 13],
 [97, 47, 29, 75, 13]]

In [168]:
temp_pages = bad_pages.copy()
middles = [swap_rules(p, rules_set) for p in temp_pages]


checking: [46, 32, 48, 27, 64, 98, 62, 99, 14]
[SWAP] X: 32, i: 1, Y: 27, j: 3
checking: [46, 27, 48, 32, 64, 98, 62, 99, 14]
[SWAP] X: 48, i: 2, Y: 32, j: 3
checking: [46, 27, 32, 48, 64, 98, 62, 99, 14]
[SWAP] X: 32, i: 2, Y: 64, j: 4
checking: [46, 27, 64, 48, 32, 98, 62, 99, 14]
[SWAP] X: 48, i: 3, Y: 32, j: 4
checking: [46, 27, 64, 32, 48, 98, 62, 99, 14]
finished: [46, 27, 64, 32, 48, 98, 62, 99, 14]
checking: [55, 89, 62, 98, 93, 51, 45, 83, 86, 23, 85, 36, 32, 43, 27, 64, 67, 46, 77, 61, 91]
[SWAP] X: 55, i: 0, Y: 83, j: 7
checking: [83, 89, 62, 98, 93, 51, 45, 55, 86, 23, 85, 36, 32, 43, 27, 64, 67, 46, 77, 61, 91]
[SWAP] X: 89, i: 1, Y: 55, j: 7
checking: [83, 55, 62, 98, 93, 51, 45, 89, 86, 23, 85, 36, 32, 43, 27, 64, 67, 46, 77, 61, 91]
[SWAP] X: 62, i: 2, Y: 98, j: 3
checking: [83, 55, 98, 62, 93, 51, 45, 89, 86, 23, 85, 36, 32, 43, 27, 64, 67, 46, 77, 61, 91]
[SWAP] X: 98, i: 2, Y: 93, j: 4
checking: [83, 55, 93, 62, 98, 51, 45, 89, 86, 23, 85, 36, 32, 43, 27, 64, 67, 46,

In [169]:
sum(middles)

4713